# IMPORTS


In [1]:
from pathlib import Path
from IPython.display import Image
import sys
from functools import cache
from utils.bayesian_optimisation import (
    HDD_PATH,
    get_candidate_solutions,
    preprocess_features,
    regression_pipeline,
    get_mean_and_std_from_model,
    expected_improvement,
    upper_confidence_bound,
    get_next_scenario_seed_from_aq,
)

sys.path.append("/home/olek/Documents/dev/metadrive-multifidelity-data/notebooks")
from utils.parse_metadrive import get_scenarios_df, process_scenario_df
import warnings
import pandas as pd
from typing import Tuple

import numpy as np

from metadrive.engine.logger import get_logger

logger = get_logger()


# LOAD DATA


In [2]:
@cache
def get_training_data():
    dir = HDD_PATH / "basic_traffic" / "0"
    scenario_file = dir / "scenarios"
    if not scenario_file.exists():
        df = get_scenarios_df(dir, multiprocessed=True)
        df.to_json(scenario_file)

    df = pd.read_json(scenario_file)
    df = process_scenario_df(df)

    df = df.set_index(["fid.ads_fps", "def.seed"]).sort_index()
    return df

# MAIN


In [3]:
np.random.seed(0)
HIGH_FIDELITY = 60

train_df = get_training_data()
candidates = get_candidate_solutions()

[INFO] Reading candidate solutions from: /media/olek/2TB_HDD/metadrive-data/candidate_solutions.json


In [4]:
X_train = preprocess_features(train_df)

In [5]:
y_train = train_df["eval.driving_score"]

In [6]:
current_best = y_train.xs(HIGH_FIDELITY).min()
logger.info(f"Current best score is: {current_best:.3f}")

[INFO] Current best score is: 0.063


In [7]:
pipe = regression_pipeline(X_train)
pipe.set_params(regressor__n_jobs=16)
model = pipe.fit(X_train, y_train)
logger.info(f"Model trained")
model

[INFO] Model trained


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('Numeric Features',
                                                  'passthrough',
                                                  Index(['fid.ads_fps', 'def.spawn_lane_index', 'def.distance', 'def.max_steps',
       'def.map_seq.1.radius', 'def.map_seq.1.angle', 'def.map_seq.1.length',
       'def.map_seq.1.dir', 'def.map_seq.1.decrease_increase',
       'def.map_seq.1.extension_length',
       ...
       'def.vehicles_data.v...
       'def.vehicles_data.vehicle_31_type',
       'def.vehicles_data.vehicle_31_spawn_road',
       'def.vehicles_data.vehicle_32_type',
       'def.vehicles_data.vehicle_32_spawn_road',
       'def.vehicles_data.vehicle_33_type',
       'def.vehicles_data.vehicle_33_spawn_road'],
      dtype='object'))])),
                ('imputer', SimpleImputer(fill_value=-1, strategy='constant')),
                ('regressor',
                 RandomForestRegressor(n_jobs=16, random_state=0))])

In [8]:
# find best candidate in high fidelity
candidates = candidates[~candidates.index.isin(train_df.index.get_level_values("def.seed"))]
logger.info(f"Considering next scenario from {len(candidates)} candidates.")

[INFO] Considering next scenario from 100000 candidates.


In [9]:
hf_test = preprocess_features(candidates)
# test candidates must be in highfidelity beacouse we want to predict hf score
hf_test["fid.ads_fps"] = HIGH_FIDELITY
hf_test = hf_test[X_train.columns]

In [10]:
mean, std = get_mean_and_std_from_model(model, hf_test)
logger.info(f"Best from model: {mean.min():.3f}")

[INFO] Best from model: 0.294


In [11]:
aq_type = "ei"
match aq_type:
    case "ei":
        aq = expected_improvement(mean, std, current_best)
    case "ucb":
        aq = upper_confidence_bound(mean, std)
    case _:
        raise ValueError("Invalid acquisition function")

[INFO] Maximum EI: -0.231
[INFO] Maximum positive EI: 0.000
[INFO] Maximum EI with uncertainty: 0.393


In [12]:
next_seed = get_next_scenario_seed_from_aq(aq, candidates)
logger.info(f"Next seed to evaluate: {next_seed}")

[INFO] Next seed to evaluate: 1024459


In [13]:
# Project next candidate to all the fidelities
next_cadidate = candidates.loc[[next_seed]]
next_cadidate

,fid.ads_fps,fid.world_fps,def.spawn_lane_index,def.distance,def.max_steps,time.init_time,time.agent_time,time.scenario_time,time.closing_time,def.map_seq.0.id,...,def.vehicles_data.vehicle_36_position_x,def.vehicles_data.vehicle_36_position_y,def.vehicles_data.vehicle_36_position_z,def.vehicles_data.vehicle_36_type,def.vehicles_data.vehicle_36_heading_theta,def.vehicles_data.vehicle_36_length,def.vehicles_data.vehicle_36_width,def.vehicles_data.vehicle_36_height,def.vehicles_data.vehicle_36_spawn_road,def.vehicles_data.vehicle_36_destination
def.seed,,,,,,,,,,,,,,,,,,,,,
1024459,60,60,0,441.520038,13246,1.688334,0,2.890000e-07,1.255474,I,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None


In [14]:
fidelity_range = [10, 20, 30, 60]
mf_candidates = pd.concat([next_cadidate] * len(fidelity_range))
mf_candidates["fid.ads_fps"] = fidelity_range

In [15]:
mf_test = mf_candidates.reset_index()[X_train.columns]
mf_test

,fid.ads_fps,def.spawn_lane_index,def.distance,def.max_steps,def.map_seq.1.radius,def.map_seq.1.angle,def.map_seq.1.length,def.map_seq.1.dir,def.map_seq.1.id,def.map_seq.1.decrease_increase,...,def.vehicles_data.vehicle_33_distance_to_ego,def.vehicles_data.vehicle_33_position_x,def.vehicles_data.vehicle_33_position_y,def.vehicles_data.vehicle_33_position_z,def.vehicles_data.vehicle_33_type,def.vehicles_data.vehicle_33_heading_theta,def.vehicles_data.vehicle_33_length,def.vehicles_data.vehicle_33_width,def.vehicles_data.vehicle_33_height,def.vehicles_data.vehicle_33_spawn_road
0,10,0,441.520038,13246,10.0,NaN,NaN,NaN,T,0.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None
1,20,0,441.520038,13246,10.0,NaN,NaN,NaN,T,0.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None
2,30,0,441.520038,13246,10.0,NaN,NaN,NaN,T,0.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None
3,60,0,441.520038,13246,10.0,NaN,NaN,NaN,T,0.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None


In [16]:
predicted_dscore, _ = get_mean_and_std_from_model(model, mf_test)

In [17]:
predictions = dict(zip(fidelity_range, predicted_dscore))
predictions

{10: np.float64(0.6321984000000002),
 20: np.float64(0.6329984000000001),
 30: np.float64(0.6424640000000004),
 60: np.float64(0.6378560000000004)}

In [18]:
hf_prediction = predictions[HIGH_FIDELITY]
hf_prediction

np.float64(0.6378560000000004)

In [19]:
# maximum absolute error
epsilon = 0.01
logger.info(str(predictions))
# go into reverse order to pick the lowest fidelity, that has acceptable error
for fid, dscore in predictions.items():
    error = abs(dscore - hf_prediction)

    logger.info(
        f"Considering {fid} FPS with predicted dscore {dscore:.3f}, error: {error:.3f}"
    )

    if error < epsilon:
        logger.info(
            f"Picking fidelity {fid} which has predicted dscore error of {error:.3f}"
        )
        break

[INFO] {10: np.float64(0.6321984000000002), 20: np.float64(0.6329984000000001), 30: np.float64(0.6424640000000004), 60: np.float64(0.6378560000000004)}
[INFO] Considering 10 FPS with predicted dscore 0.632, error: 0.006
[INFO] Picking fidelity 10 which has predicted dscore error of 0.006


# BAYESIAN OPTIMISATION ITERATION

In [ ]:
def bayes_opt_iteration(aq_type="ei", multifidelity=False) -> Tuple[int, int]:
    """
    Performs a single iteration of Bayesian Otpimisation
    Returns next scenario seed, and next fidelity to run.
    
    """
    # PREPARE TRAINING DATA
    train_df = get_training_data()
    X_train = preprocess_features(train_df)
    y_train = train_df["eval.driving_score"]

    current_best = y_train.xs(HIGH_FIDELITY).min()
    logger.info(f"Current best score is: {current_best:.3f}")

    # TRAIN THE MODEL
    pipe = regression_pipeline(X_train)
    pipe.set_params(regressor__n_jobs=16)
    model = pipe.fit(X_train, y_train)
    logger.info(f"Model trained")

    # PREPARE TEST DATA
    test_df = get_candidate_solutions()
    test_df = test_df[~test_df.index.isin(train_df.index.get_level_values("def.seed"))]
    logger.info(f"Considering next scenario from {len(test_df)} candidates.")

    X_test = preprocess_features(candidates)
    # test candidates must be in highfidelity beacouse we want to predict hf score
    X_test["fid.ads_fps"] = HIGH_FIDELITY
    X_test = hf_test[X_train.columns]

    # PREDICT DSCORE FOR HIGHFIDELITY
    dscore_predictions, std = get_mean_and_std_from_model(model, X_test)
    logger.info(f"Best from model: {dscore_predictions.min():.3f}")

    match aq_type:
        case "ei":
            aq = expected_improvement(mean, std, current_best)
        case "ucb":
            aq = upper_confidence_bound(mean, std)
        case _:
            raise ValueError("Invalid acquisition function")

    next_seed = int(get_next_scenario_seed_from_aq(aq, candidates))
    logger.info(f"Next seed to evaluate: {next_seed}")

    if not multifidelity:
        return next_seed, HIGH_FIDELITY

    logger.info(f"Multifidelity enabled")
    fidelity_range = [10, 20, 30, 60]
    next_cadidate = candidates.loc[[next_seed]]
    mf_candidates = pd.concat([next_cadidate] * len(fidelity_range))
    mf_candidates["fid.ads_fps"] = fidelity_range

    mf_X_test = mf_candidates.reset_index()[X_train.columns]
    predicted_dscore, _ = get_mean_and_std_from_model(model, mf_X_test)
    predictions = dict(zip(fidelity_range, predicted_dscore))
    hf_prediction = predictions[HIGH_FIDELITY]
    logger.info(f"Predicted dscore for high fidelity: {hf_prediction:.3f}")

    # maximum absolute error
    epsilon = 0.01
    logger.info(str(predictions))
    # go into increasing fidelity order
    for fid, dscore in predictions.items():
        error = abs(dscore - hf_prediction)
        logger.info(f"Considering {fid} FPS with predicted {dscore = :.3f}, {error = :.3f}")

        if error < epsilon:
            logger.info(f"Picking fidelity {fid} with dscore error of {error:.3f}")
            return next_seed, fid

    raise ValueError("No fidelity with acceptable error found")


bayes_opt_iteration(aq_type="ucb", multifidelity=True)

[INFO] Current best score is: 0.063
[INFO] Model trained
[INFO] Considering next scenario from 100000 candidates.
